In [1]:
%cd /Users/aravindhswaminathan/Documents/GitHub/TwoSidedYSYK
#!pwd
import numpy as np
from matplotlib import pyplot as plt
from SYK_fft import *
import sys
from scipy import optimize
from scipy.linalg import norm
import scipy.linalg as LA
from ConformalAnalytical import *
from functools import partial

/Users/aravindhswaminathan/Documents/GitHub/TwoSidedYSYK


In [2]:
Nbig = int(2**15)
err = 1e-3

beta = 1
mu = 0.0
g = 0.5
#r = 0.0 + 1e-8 * 1j
r = 1.
#r = g**2 * beta/2 + 0.001
#r = 1e-4
kappa = 1.

omega = (2 * np.arange(Nbig) - Nbig + 1) * np.pi/beta
nu = (2 * np.arange(Nbig) - Nbig ) * np.pi/beta
tau = (np.arange(Nbig) + 1./2) * beta/Nbig

In [3]:
# Gfreetau = Freq2TimeF(1./(1j*omega + mu),Nbig,beta)
# Dfreetau = Freq2TimeB(1./(nu**2 + r),Nbig,beta)
# delta = 0.420374134464041
# omegar2 = ret_omegar2(g,beta)
# Gtau_powerlaw = -1.0*np.sign(tau)*(np.pi/np.abs(beta*np.sin(np.pi*tau/beta))) ** (2*delta)
# Dtau_powerlaw =  1.0*(np.pi/np.abs(beta*np.sin(np.pi*tau/beta))) ** (2 - 4*delta)


# init_Gomega = 1./(1j*omega + mu)
# init_Domega = 1./(nu**2 + r)
# init_Greens = [init_Gomega,init_Domega]

# def fun(Greens):
#     ''' 
#     Greens[0] = Gomega, Greens[1] = Domega
#     '''
#     Gomega,Domega = Greens
#     Gtau, Dtau  = Freq2TimeF(Gomega,Nbig,beta), Freq2TimeB(Domega,Nbig,beta)
#     Sigmatau = 1.0 * kappa * (g**2) * Dtau * Gtau
#     Pitau = 2.0 * g**2 * Gtau * Gtau[::-1] 
#     Sigmaomega, Piomega = Time2FreqF(Sigmatau,Nbig,beta), Time2FreqB(Pitau,Nbig,beta)
    
#     #return np.array([Gomega - (1./(1j*omega + mu - Sigmaomega)), Domega - (1./(nu**2 + r - Piomega))])
#     return [Gomega - (1./(1j*omega + mu - Sigmaomega)), Domega - (1./(nu**2 + omegar2 - Piomega))]

# # def rmsnorm(vec):
# #     N = len(vec)
# #     return np.sqrt((1./N) * np.sum(np.abs(vec)**2))




# #Piomega[Nbig//2] = 1.0*r - omegar2
# sol = optimize.anderson(fun, init_Greens, verbose = True, f_tol = 2e-5, maxiter=100, M=4, 
#                        tol_norm = lambda vec: np.mean(norm(vec, keepdims = True, check_finite = False)))

# #sol = optimize.fsolve(fun, init_Greens)
# Gomega, Domega = sol

# Gtau, Dtau  = Freq2TimeF(Gomega,Nbig,beta), Freq2TimeB(Domega,Nbig,beta)


In [4]:
Gfreetau = Freq2TimeF(1./(1j*omega + mu),Nbig,beta)
Dfreetau = Freq2TimeB(1./(nu**2 + r),Nbig,beta)
delta = 0.420374134464041
omegar2 = ret_omegar2(g,beta)
Gtau_powerlaw = -1.0*np.sign(tau)*(np.pi/np.abs(beta*np.sin(np.pi*tau/beta))) ** (2*delta)
Dtau_powerlaw =  1.0*(np.pi/np.abs(beta*np.sin(np.pi*tau/beta))) ** (2 - 4*delta)

init_Gomega = 1./(1j*omega + mu)
init_Domega = 1./(nu**2 + r)
init_Greens = [init_Gomega,init_Domega]

def funbeta(Greens,beta_):
    ''' 
    Greens[0] = Gomega, Greens[1] = Domega
    '''
    omega_ = ImagGridMaker(Nbig,beta_,which_grid='fermion')
    nu_ = ImagGridMaker(Nbig,beta_,which_grid='boson')
    Gomega,Domega = Greens
    Gtau, Dtau  = Freq2TimeF(Gomega,Nbig,beta_), Freq2TimeB(Domega,Nbig,beta_)
    Sigmatau = 1.0 * kappa * (g**2) * Dtau * Gtau
    Pitau = 2.0 * g**2 * Gtau * Gtau[::-1] 
    Sigmaomega, Piomega = Time2FreqF(Sigmatau,Nbig,beta_), Time2FreqB(Pitau,Nbig,beta_)
    
    #return np.array([Gomega - (1./(1j*omega + mu - Sigmaomega)), Domega - (1./(nu**2 + r - Piomega))])
    return [Gomega - (1./(1j*omega_ + mu - Sigmaomega)), Domega - (1./(nu_**2 + omegar2 - Piomega))]

betalist = np.array([1,5,10,11,12,13,14])
betalist = np.append(betalist, np.arange(14.1,15,0.1))
sol = init_Greens
for loopbeta in betalist:
    print(f'started beta = {loopbeta}')
    # sol = optimize.anderson(partial(funbeta,beta_=loopbeta), sol, verbose = True, f_tol = 2e-5, maxiter=100, M=3, 
    #                    tol_norm = lambda vec: np.mean(norm(vec, keepdims = True, check_finite = False)))
    sol = optimize.anderson(partial(funbeta,beta_=loopbeta), sol, w0=1e-3 if loopbeta < 15 else 1e-4, verbose = True, f_tol = 1e-9, maxiter=1000, M=3)

#sol = optimize.fsolve(fun, init_Greens)
beta = betalist[-1]
Gomega, Domega = sol

omega = ImagGridMaker(Nbig,beta,which_grid='fermion')
nu = ImagGridMaker(Nbig,beta,which_grid='boson')
tau = ImagGridMaker(Nbig,beta,which_grid='tau')
Gtau, Dtau  = Freq2TimeF(Gomega,Nbig,beta), Freq2TimeB(Domega,Nbig,beta)


started beta = 1.0
0:  |F(x)| = 0.793451; step 1
1:  |F(x)| = 0.00734156; step 1
2:  |F(x)| = 0.000419451; step 1
3:  |F(x)| = 1.05406e-05; step 1
4:  |F(x)| = 2.95129e-06; step 1
5:  |F(x)| = 3.68572e-08; step 1
6:  |F(x)| = 4.98642e-08; step 1
7:  |F(x)| = 6.02086e-09; step 1
8:  |F(x)| = 6.95805e-09; step 1
9:  |F(x)| = 6.84716e-09; step 0.0160192
10:  |F(x)| = 3.79231e-09; step 1
11:  |F(x)| = 1.86484e-09; step 1
12:  |F(x)| = 4.70981e-09; step 1
13:  |F(x)| = 2.4884e-11; step 1
started beta = 5.0
0:  |F(x)| = 1.5867; step 1
1:  |F(x)| = 0.411932; step 1
2:  |F(x)| = 0.3994; step 1
3:  |F(x)| = 0.161369; step 0.244148
4:  |F(x)| = 0.290664; step 1
5:  |F(x)| = 0.00204781; step 1
6:  |F(x)| = 0.00234584; step 1
7:  |F(x)| = 0.00294586; step 1
8:  |F(x)| = 0.00145714; step 1
9:  |F(x)| = 0.00156548; step 1
10:  |F(x)| = 0.00169076; step 1
11:  |F(x)| = 0.00137489; step 1
12:  |F(x)| = 0.0014243; step 0.487935
13:  |F(x)| = 0.00187322; step 1
14:  |F(x)| = 0.00164771; step 1
15:  |F(x

NoConvergence: [[-9.75869120e-11+1.06857568e-04j -9.75935756e-11+1.06864090e-04j
  -9.75997313e-11+1.06870614e-04j ...  9.76009567e-11-1.06870614e-04j
   9.75945458e-11-1.06864090e-04j  9.75878215e-11-1.06857568e-04j]
 [ 1.14178446e-08+1.83315593e-14j  1.14192385e-08+1.83337973e-14j
   1.14206326e-08+1.83360357e-14j ...  1.14220271e-08+1.83382745e-14j
   1.14206326e-08+1.83360357e-14j  1.14192385e-08+1.83337973e-14j]]

In [ ]:
Gconftau = Freq2TimeF(GconfImag(omega,g,beta),Nbig,beta)
Dconftau = Freq2TimeB(DconfImag(nu,g,beta),Nbig,beta)
FreeDtau = DfreeImagtau(tau,r,beta)

fig, ax = plt.subplots(2)

ax[0].plot(tau/beta, np.real(Gtau), 'r', label = 'numerics Gtau')
ax[0].plot(tau/beta, np.real(Gconftau), 'b--', label = 'analytical Gtau' )
ax[0].set_ylim(-1,1)
ax[0].set_xlabel(r'$\tau/\beta$',labelpad = 0)
ax[0].set_ylabel(r'$\Re{G(\tau)}$')
ax[0].legend()

ax[1].plot(tau/beta, np.real(Dtau), 'r', label = 'numerics Dtau')
ax[1].plot(tau/beta, np.real(Dconftau), 'b--', label = 'analytical Dtau' )
ax[1].plot(tau/beta, np.real(FreeDtau), 'g-.', label = 'Free D Dtau' )
#ax[1].set_ylim(0,1)
ax[1].set_xlabel(r'$\tau/\beta$',labelpad = 0)
ax[1].set_ylabel(r'$\Re{D(\tau)}$')
ax[1].legend()


In [ ]:
start, stop = Nbig//2, Nbig//2 + 100
startB, stopB = Nbig//2 + 1 , Nbig//2 + 101
delta = 0.420374134464041
alt_delta = 0.116902  

fitG_val = -np.imag(Gomega[start])*(g**2)

conf_fit_G = 1 * np.abs(omega/(g**2))**(2*delta - 1)
conf_fit_G = conf_fit_G/conf_fit_G[start] * fitG_val
alt_conf_fit_G = fitG_val * np.abs(omega/(g**2))**(2*alt_delta - 1)

fitD_val = np.real(Domega[startB])*(g**2)

conf_fit_D = 1 * np.abs(nu[startB:stopB]/(g**2))**(1-4*delta)
conf_fit_D = conf_fit_D/conf_fit_D[0] * fitD_val
alt_conf_fit_D = 1 * np.abs(nu[startB]/(g**2))**(1-4*alt_delta)


fig,(ax1,ax2) = plt.subplots(1,2)
#fig.set_figwidth(10)
titlestring = r'$\beta$ = ' + str(beta) + r', $\log_2{N}$ = ' + str(np.log2(Nbig)) + r', $g = $' + str(g)
fig.suptitle(titlestring)
fig.tight_layout(pad=2)

ax1.loglog(omega[start:stop]/(g**2), -np.imag(Gomega[start:stop])*(g**2),'p',label = 'numerics')
ax1.loglog(omega[start:stop]/(g**2), conf_fit_G[start:stop],'k--',label = 'ES power law')
#ax1.loglog(omega[start:]/(g**2), -np.imag(Gconf[start:])*(g**2),'m.',label = 'ES solution')
#ax1.loglog(omega[start:]/(g**2), alt_conf_fit_G[start:],'g--', label = 'alt power law')
#ax1.set_xlim(omega[start]/2,omega[start+15])
#ax1.set_ylim(1e-1,1e1)
ax1.set_xlabel(r'$\omega_n/g^2$')
ax1.set_ylabel(r'$-g^2\,\Im{G(\omega_n)}$')
ax1.set_aspect('equal', adjustable='box')
#ax1.axis('square')
ax1.legend()


ax2.loglog(nu[startB:stopB]/(g**2), np.real(Domega[startB:stopB])*(g**2),'p',label='numerics')
ax2.loglog(nu[startB:stopB]/(g**2), conf_fit_D,'k--',label = 'ES power law')
#ax2.loglog(nu[startB:]/(g**2), np.real(Dconf[startB:]),'m.',label = 'ES solution')
#ax2.loglog(nu[startB:]/(g**2), alt_conf_fit_D,'g--', label = 'alt power law')
#ax2.set_xlim(nu[startB]/2,nu[startB+15])
#ax2.set_ylim(5e-1,100)
ax2.set_xlabel(r'$\nu_n/g^2$')
ax2.set_ylabel(r'$g^2\,\Re{D(\nu_n)}$',labelpad = None)
ax2.set_aspect('equal', adjustable='box')
ax2.legend()

#plt.savefig('lowenergy_powerlaw_ImagTime_SingleYSYK.pdf', bbox_inches = 'tight')


In [ ]:
len(fun(init_Greens))
a,b = fun(init_Greens)
a


In [ ]:
ImagGridMaker?
